[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nmickevicius/mcwBiophysicsMriCourse/blob/main/reconstructionDemo.ipynb)

# Image Reconstruction Demo 

3D balanced steady-state free precession (bSSFP) data were acquired in a phantom. The data were acquired with an acquisition matrix of 256x256x32 with a TR of 4.5 ms and flip angle of 30 degrees. Following an inverse Fourier transform along the "slice" dimension, a single slice was extracted and saved for this demo. 

## Imports and Data Download

In [ ]:
import numpy as np 
import h5py 
import matplotlib.pyplot as plt 

# download the .h5 file and load it as a complex-valued numpy array
!wget --no-check-certificate --content-disposition https://github.com/nmickevicius/mcwBiophysicsMriCourse/raw/main/data/bssfpKSpace.h5
with h5py.File('bssfpKSpace.h5','r') as F:
    data = np.array(F['data'], dtype=np.complex64)
    csm = np.array(F['csm'], dtype=np.complex64)

# Support Functions 

In [ ]:
def fft1d(inp,dim):
    return np.fft.fftshift(np.fft.fft(np.fft.fftshift(inp,dim),axis=dim),dim)

def ifft1d(inp,dim):
    return np.fft.ifftshift(np.fft.ifft(np.fft.ifftshift(inp,dim),axis=dim),dim)

def fft2d(inp):
    return fft1d(fft1d(inp,0),1)

def ifft2d(inp):
    return ifft1d(ifft1d(inp,0),1)

## Simple Inverse FFT Reconstruction

In [ ]:
# inverse FFT to get a 2D image for each coil 
rec = ifft2d(data)

# root sum-of-squares coil combination 
rss = np.sqrt(np.sum(np.abs(rec * np.conj(rec)),axis=-1))

# complex coil combination using sensitivity maps keeps the image phase! 
ccc = np.sum( rec * np.conj(csm), axis=-1)

# display RSS magnitude and phase
fig, axs = plt.subplots(1, 2, figsize=(9, 3))
magim = axs[0].imshow(np.abs(rss), cmap='gray')
plt.colorbar(magim)
phaim = axs[1].imshow(np.angle(rss), vmin=-np.pi, vmax=np.pi)
plt.colorbar(phaim)
fig.suptitle('Root Sum-of-Squares Coil Combination')

# display CCC magnitude and phase
fig, axs = plt.subplots(1, 2, figsize=(9, 3))
magim = axs[0].imshow(np.abs(ccc), cmap='gray')
plt.colorbar(magim)
phaim = axs[1].imshow(np.angle(ccc), vmin=-np.pi, vmax=np.pi)
plt.colorbar(phaim)
fig.suptitle('Complex Coil Combination')

## Undersampling Data by Factor of Two

What happens if we skipped the acquisition of every other line in k-space? 

In [ ]:
usdata = np.zeros_like(data)
usdata[:,::2,:] = data[:,::2,:]

# inverse FFT to get a 2D image for each coil 
rec = ifft2d(usdata)

# root sum-of-squares coil combination 
rss = np.sqrt(np.sum(np.abs(rec * np.conj(rec)),axis=-1))

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.log10(np.abs(usdata[:,:,0])))
plt.title('2x Accelerated k-Space')
plt.show()

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.abs(rss), cmap='gray')
plt.title('2x Accelerated Reconstruction')
plt.show()


## Now by a Factor of Four

In [ ]:
usdata = np.zeros_like(data)
usdata[:,::4,:] = data[:,::4,:]

# inverse FFT to get a 2D image for each coil 
rec = ifft2d(usdata)

# root sum-of-squares coil combination 
rss = np.sqrt(np.sum(np.abs(rec * np.conj(rec)),axis=-1))

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.log10(np.abs(usdata[:,:,0])))
plt.title('4x Accelerated k-Space')
plt.show()

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.abs(rss), cmap='gray')
plt.title('4x Accelerated Reconstruction')
plt.show()

## Random 4x Undersampling

In [ ]:
N = data.shape[1]
inds = np.random.permutation(N)[:int(N/4)]
usdata = np.zeros_like(data)
usdata[:,inds,:] = data[:,inds,:]

# inverse FFT to get a 2D image for each coil 
rec = ifft2d(usdata)

# root sum-of-squares coil combination 
rss = np.sqrt(np.sum(np.abs(rec * np.conj(rec)),axis=-1))

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.log10(np.abs(usdata[:,:,0])))
plt.title('Random 4x Accelerated k-Space')
plt.show()

fig = plt.figure() 
fig.set_figwidth(10)
fig.set_figheight(10)
plt.imshow(np.abs(rss), cmap='gray')
plt.title('Random 4x Accelerated Reconstruction')
plt.show()


## Parallel Imaging

The linear representation of the MRI signal formation process is practical for reconstructing images from undersampled measurements.

If the desired image is the vectorized $N \times N$ image given by $\mathbf{x} \in \mathbb{C}^{N^2}$, it can be related to the undersampled k-space measurements $\mathbf{y} \in \mathbb{C}^{ N_s \cdot N \cdot N_c}$ by an operator $\mathbf{A} \in \mathbb{C}^{N_s \cdot N\cdot N_c \times{N^2}}$ such that $\mathbf{y} = \mathbf{Ax}$. Here, $N_s$ is the number of phase encoding lines acquired and $N_c$ is the number of RF coils used in the measurement. 

The linear model is given by $\mathbf{A} = \mathbf{PFS}$, where $\mathbf{S}$ is a multiplication with the RF coil sensitivities, $\mathbf{F}$ is the 2D Fourier transform operator, and $\mathbf{P}$ is the k-space sampling mask. 

If $\mathbf{A}$ is practically invertible, the solution can be obtained by $\mathbf{x} = \mathbf{A}^{-1}\mathbf{y}$. However, this matrix becomes prohibitively large with increasing matrix sizes. So, this problem is often addressed iteratively. An example algorithm is shown below



In [ ]:
# function that carries out the operation A*x 
def A_for(x, S, P):
    return P * fft2d(S*x[:,:,None])

# function that carries out the operation A'*y 
def A_adj(y, S, P):
    return np.sum(np.conj(S) * ifft2d(P*y), axis=-1)

# SENSE reconstruction solved with conjugate gradient methods
def sense_cg(y, S, tik=0.0, niters=40):

    # get sampling mask 
    P = np.zeros_like(y)
    P[np.abs(y) > 0.0] = 1.0

    # function that carries out the normal operator A'*A + tik*I 
    AhA = lambda x: A_adj(A_for(x, S, P), S, P) + tik*x 

    x = np.zeros((y.shape[0], y.shape[1]), dtype=y.dtype)
    b = A_adj(y, S, P)
    r = b - AhA(x)
    p = np.copy(r)
    rsnot = np.sum(r*np.conj(r))
    rsold = rsnot 
    rsnew = rsnot 
    for i in range(niters):
        Ap = AhA(p) 
        pAp = np.sum(p*np.conj(Ap))
        alpha = rsold / pAp 
        x = x + alpha*p 
        r = r - alpha*Ap 
        rsnew = np.sum(r*np.conj(r))
        beta = rsnew/rsold 
        rsold = rsnew 
        p = beta*p + r
        
    return x


## SENSE Reconstruction of 2x Random Sampling

In [ ]:
N = data.shape[1]
inds = np.random.permutation(N)[:int(N/2)]
usdata = np.zeros_like(data)
usdata[:,inds,:] = data[:,inds,:]

# simple IFFT reconstruction
rec_ifft = ifft2d(usdata)
rec_ifft = np.sqrt(np.sum(rec_ifft * np.conj(rec_ifft), axis=-1))

# SENSE reconstruction 
rec_sense = sense_cg(usdata, csm, niters=20)

fig = plt.figure() 
fig.set_figwidth(5)
fig.set_figheight(5)
plt.imshow(np.abs(rec_ifft),cmap='gray')
plt.title('IFFT')
plt.show()

fig = plt.figure() 
fig.set_figwidth(5)
fig.set_figheight(5)
plt.imshow(np.abs(rec_sense),cmap='gray')
plt.title('SENSE')
plt.show()